### Подход 1: градиентный бустинг "в лоб"

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from datetime import datetime
import pandas as p
from numpy import average
import sklearn
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

In [4]:
features = p.read_csv('./features.csv', index_col='match_id')
features_test = p.read_csv('./features_test.csv', index_col='match_id')
features_train = features.drop(columns=[
     "duration", 
     "radiant_win",
     "tower_status_radiant",
     "tower_status_dire",
     "barracks_status_dire",
     "barracks_status_radiant"
])

In [5]:
values_count = features_train.count().sort_values(ascending=True)

#### Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [6]:
rows = values_count.max()

print("Количество пропусков\n")
for (feature, val) in sorted(values_count.iteritems()):
    values_missing = rows - val
    if values_missing > 0:
        print(f"{feature}: {values_missing}")

Количество пропусков

dire_bottle_time: 16143
dire_courier_time: 676
dire_first_ward_time: 1826
dire_flying_courier_time: 26098
first_blood_player1: 19553
first_blood_player2: 43987
first_blood_team: 19553
first_blood_time: 19553
radiant_bottle_time: 15691
radiant_courier_time: 692
radiant_first_ward_time: 1836
radiant_flying_courier_time: 27479


Много значений пропущено в колонках, касающихся First Blood (first_blood_player1, first_blood_player2, first_blood_time, first_blood_team). Это значит, что игроки никого не убили за первые 5 минут матча.

Пропуски в *_bottle_time, *_courier_time и *_flying_courier_time означают, что соответствующая команда не купила эти предметы за первые 5 минут.

Пропуски в *_first_ward_time означают, что команда не поставила ни одного наблюдателя за 5 минут.

In [7]:
features_train = features_train.fillna(value=0)

#### 2. Как называется столбец, содержащий целевую переменную?

In [8]:
Y_col = "radiant_win"


X = features_train
y = features[Y_col]

#### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.

In [9]:
n_splits = 5
kfold = KFold(shuffle=True, n_splits=n_splits)

Обучается классификатор с 10 деревьями
Среднее значения качества: 0.6646291455551827 (за 0:01:12.878092мс)

Обучается классификатор с 20 деревьями
Среднее значения качества: 0.6809860273308972 (за 0:02:18.501172мс)

**Обучается классификатор с 30 деревьями
Среднее значения качества: 0.6900458742897161 (за 0:03:09.979634мс)**

Обучается классификатор с 40 деревьями
Среднее значения качества: 0.694294321131233 (за 0:04:12.675311мс)

Обучается классификатор с 50 деревьями
Среднее значения качества: 0.6977442051399642 (за 0:05:26.558180мс)

Обучается классификатор с 60 деревьями
Среднее значения качества: 0.7002027254594443 (за 0:06:49.034804мс)

Обучается классификатор с 70 деревьями
Среднее значения качества: 0.7019988271689213 (за 0:07:50.472654мс)

Обучается классификатор с 80 деревьями
Среднее значения качества: 0.7047217159467425 (за 0:09:17.449093мс)

Обучается классификатор с 90 деревьями
Среднее значения качества: 0.7058723237655931 (за 0:09:42.171533мс)

Обучается классификатор с 100 деревьями
Среднее значения качества: 0.7069479623025787 (за 0:11:31.993495мс)

#### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

При количества деревьев больше 30 качество продолжает расти, не медленее. При 80+ рост замедляется ещё сильнее. К тому же, время на обучение начинает заметно увеличиваться.
Это значит, что оптимально испольховать 60 деревьев.

Чтобы ускорить обучение, можно использовать метод главных компонент.

### Подход 2: логистическая регрессия

Отмасштабируем признаки

In [10]:
from sklearn.preprocessing import StandardScaler 
import numpy as np

scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X.astype(np.float64))
X_scaled = p.DataFrame(data=X_scaled_array, index=X.index, columns=X.columns)

**1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**

In [44]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA 
from numpy import linspace
from scipy.special import expit
from mpl_toolkits.mplot3d import Axes3D 

def _plot_(c, x, clf, feature_selector='coef'):
    n_features = 2 # always plotting in 3D
    y_pred = clf.predict_proba(x)[:, 1]
    if feature_selector == 'pca':
        ### PCA
        pca = PCA(n_components=n_features)
        axis = p.DataFrame(pca.fit_transform(x, y))
        labels = None
    elif feature_selector == 'coef':
        ### Coefs (feature importance)
        features = p.DataFrame(zip(clf.coef_[0], x), columns = ["coef", "column"])
        important_features = features.nlargest(n_features, "coef")
        labels = important_features['column'].values
        axis = x[important_features['column']]
    else:
        raise ValueError("Unknown feature selector")
        
    # Data 
    axis = np.column_stack([y_pred, axis.values])
    axis = p.DataFrame(axis)
    radiant = axis.loc[y.loc[y>=0.5].index]
    dire = axis.loc[y.loc[y<0.5].index]
    colors = ["red" if y>=0.5 else "green" for y in y_pred]
    
    # 3D plotting
    fig = plt.figure()
    fig.suptitle(f"C={c}")
    ax = plt.axes(projection='3d')
    ax.set_xlabel(labels[1])
    ax.set_ylabel(Y_col)
    ax.set_zlabel(labels[0])
    ax.scatter3D(axis.iloc[:, 2], axis.iloc[:, 0], axis.iloc[:, 1], c=colors)
    # ax.scatter3D(dire.iloc[:, 1], dire.iloc[:, 0], dire.iloc[:, 2], label="Dire", c="red")
    # ax.scatter3D(radiant.iloc[:, 1], radiant.iloc[:, 0], radiant.iloc[:, 2], label="Radiant", c="green")

    # 2d
    # plt.scatter(axis.iloc[:, 1], axis.iloc[:, 0], c=colors)
    # plt.scatter(dire.iloc[:, 0], dire.iloc[:, 1], label="Dire", c="red")
    # plt.scatter(radiant.iloc[:, 0], radiant.iloc[:, 1], label="Radiant", c="green")

    # Loss
    # loss = expit(x.values * clf.coef_ + clf.intercept_)
    # plt.plot(np.array(x), loss, color='black', linewidth=3)

    plt.show()
    
def _fit_and_score_(x, c):
    ll_clf = LogisticRegressionCV(cv=kfold, penalty='l2')
    ll_clf.scoring = "roc_auc"
    ll_clf.C = c
    ll_clf.fit(x, y)
    return ll_clf, ll_clf.score(x, y)
    
def fit_log_reg(x):
    best_score = None
    best_c = None
    best_clf = None
    for c in linspace(0.0001, 10, num=1):
        start_time = datetime.now()
        print(f"Обучается LogisticRegression с C={c}")
        ll_clf, avg_score = _fit_and_score_(x, c)
        if best_score is None or avg_score > best_score:
            best_score = avg_score
            best_c = c
            best_clf = ll_clf
        exec_time = datetime.now() - start_time    
        print(f"Средний AUC-ROC = {avg_score} найден за {exec_time}мс")
    _plot_(c, x, best_clf)
    return best_clf, best_score, best_c

In [ ]:
clf, score, c = fit_log_reg(X_scaled)
print(f"\n🏆 Лучшее качество логистической регрессии = {score} (при c={c})")

Обучается LogisticRegression с C=0.0001


🏆 Лучшее качество логистической регрессии = 0.7179728937697301 (при c=10.0)

Качество регрессии получилось примерно таким же, как при бустинге (~0.7). Я думаю, это обусловлено тем, что я никак не изменил датасет (не считая масштабирования). Но регрессия работает примерно в 20 раз быстрее бустинга — около 20 секунд против 7 минут.

**2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?**

Уберем категориальные признаки из выборки

In [ ]:
from itertools import chain

categorial_features = list(chain.from_iterable((f"r{i}_hero", f"d{i}_hero") for i in range(1, 6))) # heroes
categorial_features += ["lobby_type"]

In [ ]:
X_with_no_categorial_features = X_scaled.filter([x for x in X_scaled.columns if x not in categorial_features])

Обучим регрессию на выборке без категориальных признаков

In [ ]:
best_clf, score, c = fit_log_reg(X_with_no_categorial_features)
print(f"🏆 Лучшее качество на выборке без категориальных признаков: {score} (c={c})")

🏆 Лучшее качество на выборке без категориальных признаков: 0.717832161391986 (c=10.0)

Качество получилось чуть хуже, чем на выборке с категориальными признаками (0.7178). Возможно, оно не изменилось потому, что эти признаки не влияли на результат.

**3. Сколько различных идентификаторов героев существует в данной игре?**

In [ ]:
heroes_dict = p.read_csv("data/dictionaries/heroes.csv")
heroes_count = heroes_dict.shape[0] # 112
heroes_count

Сформируем мешок слов

In [ ]:
heroes = np.zeros((X.shape[0], heroes_count))

for i, match_id in enumerate(X.index):
    for n in range(4):
        r_hero = X.ix[match_id, f'r{n+1}_hero'] - 1
        d_hero = X.ix[match_id, f'd{n+1}_hero'] - 1
        heroes[i, r_hero] = 1
        heroes[i, d_hero] = -1

Добавим его в выборку

In [ ]:
df_heroes = p.DataFrame(heroes, columns=heroes_dict['name'])
df_heroes = df_heroes.reset_index(drop=True)
X_with_no_categorial_features = X_with_no_categorial_features.reset_index(drop=True)
X_with_heroes = X_with_no_categorial_features.join(df_heroes)

**4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?**

In [ ]:
best_clf, score, c = fit_log_reg(X_with_heroes)
print(f"🏆 Лучшее качество на выборке с мешком слов: {score} (c={c})")

🏆 Лучшее качество на выборке с мешком слов: 0.746641390964292 (c=5.00005)

Качество улучшилось, потому что метки героев стали осмысленными, а не просто случайными идентификаторами. 

**5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?**